In [285]:
import pandas as pd, numpy as np, requests

dmg_spells = pd.read_html('https://listfist.com/list-of-dungeons-dragons-5e-spells-by-damage')[0]

In [26]:
dmg_spells

,L,Spell Name,Spell Save,Damage,Damage Type,Damage Increases at Higher Level,Upcast
0,0,Acid Splash,Dexterity,1d6,Acid,✓,✗
1,0,Booming Blade,✗,1d8,Thunder,✓,✗
2,0,Chill Touch,✗,1d8,Necrotic,✓,✗
3,0,Create Bonfire,Dexterity,1d8,Fire,✓,✗
4,0,Eldritch Blast,✗,1d10,Force,✓,✗
...,...,...,...,...,...,...,...
171,9,Psychic Scream,Intelligence,14d6,Psychic,✗,✗
172,9,Ravenous Void,Strength,5d10,Force,✗,✗
173,9,Storm of Vengeance,Constitution,2d6 / 1d6 / 10d6 / 2d6 / 1d6,"Acid, Bludgeoning, Cold, Lightning, Thunder",✗,Dice
174,9,Time Ravage,Constitution,10d12,Necrotic,✗,✗


In [286]:
dmg_spells_copy = dmg_spells.copy()
dmg_spells_copy.loc[dmg_spells_copy['Damage Increases at Higher Level'] == '✓','Damage Increases at Higher Level'] = True
dmg_spells_copy.loc[dmg_spells_copy['Damage Increases at Higher Level'] == '✗','Damage Increases at Higher Level'] = False
dmg_spells_copy.loc[dmg_spells_copy['Upcast'] == '✗','Upcast'] = None
dmg_spells_copy.loc[dmg_spells_copy['Spell Save'] == '✗','Spell Save'] = None
dmg_spells_copy['Spell Save'] = np.select(
    [
        dmg_spells_copy['Spell Save']=='Dexterity',
        dmg_spells_copy['Spell Save']=='Constitution',
        dmg_spells_copy['Spell Save']=='Strength',
        dmg_spells_copy['Spell Save']=='Intelligence',
        dmg_spells_copy['Spell Save']=='Wisdom',
        dmg_spells_copy['Spell Save']=='Charisma'
    ],
    [
        'DEX','CON','STR','INT','WIS','CHR'
    ],
    default=None
)
dmg_spells_copy.columns = [
    'Lvl','Name','Save','Dmg','DmgType','HigherLvlIncrease','Upcast'
]
dmg_spells_copy['dmgCpy'] = dmg_spells_copy['Dmg'].copy()
# dmg_spells_copy['dmg_die']
dmg_spells_copy
# dmg_spells_copy.columns


,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy
0,0,Acid Splash,DEX,1d6,Acid,True,None,1d6
1,0,Booming Blade,None,1d8,Thunder,True,None,1d8
2,0,Chill Touch,None,1d8,Necrotic,True,None,1d8
3,0,Create Bonfire,DEX,1d8,Fire,True,None,1d8
4,0,Eldritch Blast,None,1d10,Force,True,None,1d10
...,...,...,...,...,...,...,...,...
171,9,Psychic Scream,INT,14d6,Psychic,False,None,14d6
172,9,Ravenous Void,STR,5d10,Force,False,None,5d10
173,9,Storm of Vengeance,CON,2d6 / 1d6 / 10d6 / 2d6 / 1d6,"Acid, Bludgeoning, Cold, Lightning, Thunder",False,Dice,2d6 / 1d6 / 10d6 / 2d6 / 1d6
174,9,Time Ravage,CON,10d12,Necrotic,False,None,10d12


In [24]:
from bs4 import BeautifulSoup as BS
import re 
dmg_spells_copy['desc'] = ''
descs = []
ucs = []
lvlInc = []
for i,row in dmg_spells_copy.iterrows():
    uc = None
    out = None
    try:
        f = open(
            './data_for_parsing/spells/{}.html'.format(row['Name']),
            'r'
        ).read()
    except:
        print('file not found: {}'.format(
            './data_for_parsing/spells/{}.html'.format(row['Name'])
        ))
        pass
    print(row['Name'])
    soup = BS(f)
    #parse the base damage...
    body = soup(id=re.compile(r'page-content*'))[0]
    description = re.findall('(?<=Duration:).*?(?=\<em\>)',body.decode_contents(),re.DOTALL)
    if len(description) > 0:
        desc = list.copy(description)
        desc = desc[0]
        desc = desc[desc.index('<p>'):]
        p = re.compile(r'<.*?>')
        desc = re.sub(p,' ',desc)
        # desc = desc[desc.index('\n'):]
        desc = re.sub(r'\s+', ' ', desc)
        descs.append(desc)

    higherLvls = re.findall(
            '(?<=At Higher Levels\.).*?(?=\</p\>)',
            body.decode_contents(),
            re.DOTALL
    )
    deals_damage = re.findall('(?<=take|deal).*?(?=damage)',body.decode_contents())
    print("\t{}".format(deals_damage))

    if row['HigherLvlIncrease']:
        #parse the increase
        try:
            chrLvlInc = re.findall('(?<=when you reach).*?(?=\n)',higherLvls[0])
            chrLvlInc2 = re.findall('(\d{1,2}[a-zA-Z]{2}) level \((\dd\d)\)',chrLvlInc[0])
            if len(chrLvlInc2) > 0:
                chrLvls = [int(x[0][:-2]) for x in chrLvlInc2]
                dmgDice = [tuple(x[1].split('d')) for x in chrLvlInc2]
                dmgDice = [{int(x):int(y)} for x,y in dmgDice]
                out = [{a:b} for a,b in zip(chrLvls,dmgDice)]
                # if len(out) > 0:
                #     lvlInc.append(out)
                # else:
                #     lvlInc.append(None)
                print("\tChar Level Increases: {}".format(out))
        except:
            out = None

    if row['Upcast']:
        try:
            upcast = re.findall(
                re.compile(r'of \d{1,2}[a-z]{2} level or higher.*?(?=for each)'),
                higherLvls[0]
            )
            min_level = re.findall('(?<=of )(\d+[a-zA-Z]{2})(?= level)',upcast[0])
            min_level = min_level[0][:-2]
            inc_by = re.findall('(?<=increase).*',upcast[0])
            if len(inc_by) > 0:
                inc_by = re.findall('(?<=by ).*',inc_by[0])
                # inc_by = inc_by[0]
                inc_by = tuple(inc_by[0].split('d'))
                # inc_by = {inc_by[0]:inc_by[1]}
                # inc_by = [tuple(x)]
                inc_by = {int(inc_by[0]):int(inc_by[1])}
            # if len(inc_by) > 0:
            #     inc_by = inc_by[0]
            uc = {int(min_level):inc_by}
            print("\tMin Level Increase: {}\n\tUpcast:{}\n\tInc By:{}\n\tuc:{}".format(
                    min_level,
                    upcast,
                    inc_by,
                    uc
                )
            )
        except:
            uc = None         
        
    if row['Save']:
        pass

    ucs.append(uc)
    lvlInc.append(out)

dmg_spells_copy['desc'] = descs
dmg_spells_copy['upcast'] = ucs
dmg_spells_copy['lvlInc'] = lvlInc
dmg_spells_copy

Acid Splash
	[' 1d6 acid ']
	Char Level Increases: [{5: {2: 6}}, {11: {3: 6}}, {17: {4: 6}}]
Booming Blade
	['s 1d8 thunder ', 's an extra 1d8 thunder ', 's for moving increases to 2d8. Both ']
Chill Touch
	['s 1d8 necrotic ']
	Char Level Increases: [{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]
Create Bonfire
	[' 1d8 fire ']
	Char Level Increases: [{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]
Eldritch Blast
	['s 1d10 force ']
Fire Bolt
	['s 1d10 fire ']
Frostbite
	['s 1d6 cold ']
	Char Level Increases: [{5: {2: 6}}, {11: {3: 6}}, {17: {4: 6}}]
Green-Flame Blade
	['s fire ', 's an extra 1d8 fire ']
Infestation
	['s 1d6 poison ']
	Char Level Increases: [{5: {2: 6}}, {11: {3: 6}}, {17: {4: 6}}]
Lightning Lure
	[' 1d8 lightning ']
	Char Level Increases: [{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]
Magic Stone
	['s bludgeoning ']
Mind Sliver
	[' 1d6 psychic ']
	Char Level Increases: [{5: {2: 6}}, {11: {3: 6}}, {17: {4: 6}}]
Poison Spray
	[' 1d12 poison ']
Primal Savagery
	['s 1d10 acid ']
Produce Flam

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy,desc,upcast,lvlInc
0,0,Acid Splash,DEX,1d6,Acid,True,None,1d6,You hurl a bubble of acid. Choose one creatur...,None,"[{5: {2: 6}}, {11: {3: 6}}, {17: {4: 6}}]"
1,0,Booming Blade,None,1d8,Thunder,True,None,1d8,You brandish the weapon used in the spell’s c...,None,None
2,0,Chill Touch,None,1d8,Necrotic,True,None,1d8,"You create a ghostly, skeletal hand in the sp...",None,"[{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]"
3,0,Create Bonfire,DEX,1d8,Fire,True,None,1d8,You create a bonfire on ground that you can s...,None,"[{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]"
4,0,Eldritch Blast,None,1d10,Force,True,None,1d10,A beam of crackling energy streaks toward a c...,None,None
...,...,...,...,...,...,...,...,...,...,...,...
171,9,Psychic Scream,INT,14d6,Psychic,False,None,14d6,You unleash the power of your mind to blast t...,None,None
172,9,Ravenous Void,STR,5d10,Force,False,None,5d10,You create a 20-foot-radius sphere of destruc...,None,None
173,9,Storm of Vengeance,CON,2d6 / 1d6 / 10d6 / 2d6 / 1d6,"Acid, Bludgeoning, Cold, Lightning, Thunder",False,Dice,2d6 / 1d6 / 10d6 / 2d6 / 1d6,"A churning storm cloud forms, centered on a p...",None,None
174,9,Time Ravage,CON,10d12,Necrotic,False,None,10d12,You target a creature you can see within rang...,None,None


In [4]:
dmg_spells_copy.loc[dmg_spells_copy['Dmg'].str.contains(r'\+',regex=True)]

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy,desc,upcast,lvlInc
10,0,Magic Stone,None,1d6+SAM,Bludgeoning,True,None,1d6+SAM,You touch one to three pebbles and imbue them...,None,None
18,0,Shillelagh,None,1d8+SAM,Bludgeoning,False,None,1d8+SAM,The wood of a club or quarterstaff you are ho...,None,None
31,1,Chaos Bolt,None,2d8+1d6,Variable,False,Dice,2d8+1d6,"You hurl an undulating, warbling mass of chao...",{2: []},None
46,1,Magic Missile,None,1d4+1 (x3),Force,False,Number,1d4+1 (x3),You create three glowing darts of magical for...,{2: []},None
75,2,Spiritual Weapon,None,1d8+SAM,Force,False,Dice,1d8+SAM,"You create a floating, spectral weapon within...",None,None
106,4,Ice Storm,DEX,2d8+4d6,"Bludgeoning, Cold",False,Dice,2d8+4d6,A hail of rock-hard ice pounds to the ground ...,{5: {1: 8}},None
117,5,Bigby's Hand,None,4d8 / 2d6+SAM,"Bludgeoning, Force",False,Dice,4d8 / 2d6+SAM,"You create a Large hand of shimmering, transl...",None,None
121,5,Destructive Wave,CON,5d6+5d6,"Thunder, Variable",False,None,5d6+5d6,"You strike the ground, creating a burst of di...",None,None
123,5,Flame Strike,DEX,4d6+4d6,"Fire, Radiant",False,Dice,4d6+4d6,A vertical column of divine fire roars down f...,{6: {1: 6}},None
138,6,Disintegrate,DEX,10d6+40,Force,False,Dice,10d6+40,A thin green ray springs from your pointing f...,{7: {3: 6}},None


In [7]:
dmg_spells_copy['mod'] = False
for i, row in dmg_spells_copy.iterrows():
    if '+SAM' in row['Dmg']:
        dmg_spells_copy.loc[i,'mod'] = True
        dmg_spells_copy.loc[i,'Dmg'] = dmg_spells_copy.loc[i,'Dmg'][:-4]
dmg_spells_copy

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy,desc,upcast,lvlInc,mod
0,0,Acid Splash,DEX,1d6,Acid,True,None,1d6,You hurl a bubble of acid. Choose one creatur...,None,"[{5: {2: 6}}, {11: {3: 6}}, {17: {4: 6}}]",False
1,0,Booming Blade,None,1d8,Thunder,True,None,1d8,You brandish the weapon used in the spell’s c...,None,None,False
2,0,Chill Touch,None,1d8,Necrotic,True,None,1d8,"You create a ghostly, skeletal hand in the sp...",None,"[{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]",False
3,0,Create Bonfire,DEX,1d8,Fire,True,None,1d8,You create a bonfire on ground that you can s...,None,"[{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]",False
4,0,Eldritch Blast,None,1d10,Force,True,None,1d10,A beam of crackling energy streaks toward a c...,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...
171,9,Psychic Scream,INT,14d6,Psychic,False,None,14d6,You unleash the power of your mind to blast t...,None,None,False
172,9,Ravenous Void,STR,5d10,Force,False,None,5d10,You create a 20-foot-radius sphere of destruc...,None,None,False
173,9,Storm of Vengeance,CON,2d6 / 1d6 / 10d6 / 2d6 / 1d6,"Acid, Bludgeoning, Cold, Lightning, Thunder",False,Dice,2d6 / 1d6 / 10d6 / 2d6 / 1d6,"A churning storm cloud forms, centered on a p...",None,None,False
174,9,Time Ravage,CON,10d12,Necrotic,False,None,10d12,You target a creature you can see within rang...,None,None,False


## Edge Case - Multiple Damage Types "/"

In [8]:
import re #need to also handle upcast and/or levelInc columns, too.
dmg_spells_copy['dmgDict'] = None
dmg_spells_copy['dmgDict'] = dmg_spells_copy['dmgDict'].astype('object')
all_dmg = []
for i,row in dmg_spells_copy.loc[dmg_spells_copy['Dmg'].str.contains(r'/',regex=True)].iterrows():
    # print(i,row)
    try:
        f = open(
            './data_for_parsing/spells/{}.html'.format(row['Name']),
            'r'
        ).read()
    except:
        print('file not found: {}'.format(
            './data_for_parsing/spells/{}.html'.format(row['Name'])
        ))
        pass
    rowDmg = []
    for dmg in row['DmgType'].split(','):
        val = re.findall('(\d+d\d+).*{} damage'.format(dmg.lower()),f,re.IGNORECASE)
        val = list(set(val))
        d = {}
        if len(val) > 0:
            val = val[0].split('d')
            d = {int(val[0]):int(val[1])}
        if len(d.keys()) > 0:
            d = {dmg.strip():d}
        else:
            d = {dmg.strip():{}}
        # print("{}: {} {}".format(row['Name'],val,d,dmg))
        rowDmg.append(d)
    try:
        dmg_spells_copy.at[i,'dmgDict'] = rowDmg
        print(rowDmg)
    except Exception as e:
        print("error - couldn't add to row: {}".format(rowDmg))
        print(e)
    all_dmg.append(rowDmg)


# all_dmg

dmg_spells_copy.loc[dmg_spells_copy['Dmg'].str.contains(r'/',regex=True)]

    ## extract out the data from the file where we know there may be damage 
    # for dmg in row['Dmg'].split('/'):
    #     # print(f'{dmg.strip()}.*?(?= damage)')
    #     matches = re.findall(f'{dmg.strip()}.*?(?= damage)',f)
    #     print('{}\n\tdamage:{}'.format(row['Name'],matches))


[{'Necrotic': {1: 8}}]
[{'Cold': {1: 10}}, {'Piercing': {1: 10}}]
[{'Fire': {1: 6}}]
file not found: ./data_for_parsing/spells/Maximilian's Earthen Grasp.html
[{'Bludgeoning': {}}]
[{'Acid': {4: 4}}]
[{'Acid': {2: 6}}, {'Cold': {2: 6}}]
[{'Lightning': {2: 8}}]
[{'Bludgeoning': {2: 6}}, {'Lightning': {4: 6}}]
[{'Acid': {10: 4}}]
[{'Bludgeoning': {}}, {'Force': {4: 8}}]
[{'Radiant': {2: 8}}]
[{'Fire': {7: 6}}]
[{'Radiant': {4: 8}}]
[{'Bludgeoning': {3: 8}}, {'Slashing': {4: 6}}]
[{'Fire': {1: 10}}]
[{'Cold': {10: 6}}]
[{'Bludgeoning': {5: 10}}]
[{'Acid': {10: 6}}, {'Cold': {10: 6}}, {'Fire': {10: 6}}, {'Lightning': {10: 6}}, {'Poison': {10: 6}}]
[{'Acid': {1: 6}}, {'Bludgeoning': {2: 6}}, {'Cold': {1: 6}}, {'Lightning': {10: 6}}, {'Thunder': {2: 6}}]


,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy,desc,upcast,lvlInc,mod,dmgDict
23,0,Toll the Dead,WIS,1d8 / 1d12,Necrotic,True,None,1d8 / 1d12,You point at one creature you can see within ...,None,None,False,[{'Necrotic': {1: 8}}]
43,1,Ice Knife,DEX,1d10 / 2d6,"Cold, Piercing",False,Dice,1d10 / 2d6,You create a shard of ice and fling it at one...,{2: {1: 6}},None,False,"[{'Cold': {1: 10}}, {'Piercing': {1: 10}}]"
49,1,Searing Smite,CON,1d6 / 1d6,Fire,False,Dice,1d6 / 1d6,The next time you hit a creature with a melee...,{2: {1: 6}},None,False,[{'Fire': {1: 6}}]
65,2,Maximilian's Earthen Grasp,STR,2d6 / 2d6,Bludgeoning,False,None,2d6 / 2d6,"Choose a manufactured metal object, such as a...",None,None,False,[{'Bludgeoning': {}}]
66,2,Melf's Acid Arrow,None,4d4 / 2d4,Acid,False,Dice,4d4 / 2d4,A shimmering green arrow streaks toward a tar...,{3: {1: 4}},None,False,[{'Acid': {4: 4}}]
87,3,Hunger of Hadar,DEX,2d6 / 2d6,"Acid, Cold",False,None,2d6 / 2d6,You open a gateway to the dark between the st...,None,None,False,"[{'Acid': {2: 6}}, {'Cold': {2: 6}}]"
89,3,Lightning Arrow,DEX,4d8 / 2d8,Lightning,False,Dice,4d8 / 2d8,The next time you make a ranged weapon attack...,{4: {1: 8}},None,False,[{'Lightning': {2: 8}}]
113,4,Storm Sphere,STR,2d6 / 4d6,"Bludgeoning, Lightning",False,None,2d6 / 4d6,A 20-foot-radius sphere of whirling air sprin...,None,None,False,"[{'Bludgeoning': {2: 6}}, {'Lightning': {4: 6}}]"
114,4,Vitriolic Sphere,DEX,10d4 / 5d4,Acid,False,Dice,10d4 / 5d4,"You point at a place within range, and a glow...",{5: {2: 4}},None,False,[{'Acid': {10: 4}}]
117,5,Bigby's Hand,None,4d8 / 2d6,"Bludgeoning, Force",False,Dice,4d8 / 2d6+SAM,"You create a Large hand of shimmering, transl...",None,None,True,"[{'Bludgeoning': {}}, {'Force': {4: 8}}]"


## Edge Case - Addition of Damage "+"

In [20]:
dmg_spells_copy.loc[dmg_spells_copy['Dmg'].str.contains(r'\+',regex=True)]
#if SAM in Dmg => spellcast modifier addition
#if \+\d+(?!=d) in Dmg => hard addition
# if (x\d+) in Dmg => multiple casts
# otherwise it's just \d+d\d+ \+ \d+\d\d+ with two damage types
dmg_spells_copy[['multDmg','hardAdd','addSCMod','multiCast']] = None,None,None,None
for i,row in dmg_spells_copy.loc[dmg_spells_copy['Dmg'].str.contains(r'\+',regex=True)].iterrows():
    try:
        f = open(
            './data_for_parsing/spells/{}.html'.format(row['Name']),
            'r'
        ).read()
    except:
        print('file not found: {}'.format(
            './data_for_parsing/spells/{}.html'.format(row['Name'])
        ))
        pass
    if "SAM" in row['Dmg']:
        pass
    hard_add = re.findall('\+\d+(?!d)',row['Dmg'])
    if len(hard_add) > 0:
        hard_add = int(hard_add[0][1:])
    mod = re.findall('SAM',row['Dmg'])
    mod = len(mod) > 0
    mult = re.findall('(?<=\(x)\d+',row['Dmg'])
    dmgs = re.findall('(\d+d\d+)\+(\d+d\d+)',row['Dmg'])
    if not mod and hard_add is not None:
        dmgs = re.findall('\d+d\d+',row['Dmg'])
    print(row['Dmg'],":",dmgs,hard_add,mod,mult)
    dmg_spells_copy.at[i,'multDmg'] = dmgs
    dmg_spells_copy.at[i,'hardAdd'] = hard_add
    dmg_spells_copy.at[i,'addSCMod'] = mod
    dmg_spells_copy.at[i,'multiCast'] = mult
dmg_spells_copy.loc[dmg_spells_copy['Dmg'].str.contains('\+')]

2d8+1d6 : ['2d8', '1d6'] [] False []
1d4+1 (x3) : ['1d4'] 1 False ['3']
2d8+4d6 : ['2d8', '4d6'] [] False []
5d6+5d6 : ['5d6', '5d6'] [] False []
4d6+4d6 : ['4d6', '4d6'] [] False []
10d6+40 : ['10d6'] 40 False []
7d8+30 : ['7d8'] 30 False []
20d6+20d6 : ['20d6', '20d6'] 2 False []


,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,desc,upcast,lvlInc,mod,dmgDict,multDmg,hardAdd,addSCMod,multiCast,dmg_die_count,dmg_die_size,uc
31,1,Chaos Bolt,None,2d8+1d6,Variable,False,Dice,"You hurl an undulating, warbling mass of chao...",{2: []},None,False,None,"[2d8, 1d6]",[],False,[],None,None,True
46,1,Magic Missile,None,1d4+1 (x3),Force,False,Number,You create three glowing darts of magical for...,{2: []},None,False,None,[1d4],1,False,[3],1,4,True
106,4,Ice Storm,DEX,2d8+4d6,"Bludgeoning, Cold",False,Dice,A hail of rock-hard ice pounds to the ground ...,{5: {1: 8}},None,False,None,"[2d8, 4d6]",[],False,[],None,None,True
121,5,Destructive Wave,CON,5d6+5d6,"Thunder, Variable",False,None,"You strike the ground, creating a burst of di...",None,None,False,None,"[5d6, 5d6]",[],False,[],None,None,False
123,5,Flame Strike,DEX,4d6+4d6,"Fire, Radiant",False,Dice,A vertical column of divine fire roars down f...,{6: {1: 6}},None,False,None,"[4d6, 4d6]",[],False,[],None,None,True
138,6,Disintegrate,DEX,10d6+40,Force,False,Dice,A thin green ray springs from your pointing f...,{7: {3: 6}},None,False,None,[10d6],40,False,[],10,6,True
153,7,Finger of Death,CON,7d8+30,Necrotic,False,None,You send negative energy coursing through a c...,None,None,False,None,[7d8],30,False,[],7,8,False
169,9,Meteor Swarm,DEX,20d6+20d6,"Bludgeoning, Fire",False,None,Blazing orbs of fire plummet to the ground at...,None,None,False,None,"[20d6, 20d6]",2,False,[],None,None,False


In [10]:
for i,row in dmg_spells_copy.loc[dmg_spells_copy['Dmg'].str.contains(r'\+',regex=True)].iterrows():
    try:
        f = open(
            './data_for_parsing/spells/{}.html'.format(row['Name']),
            'r'
        ).read()
    except:
        print('file not found: {}'.format(
            './data_for_parsing/spells/{}.html'.format(row['Name'])
        ))
        pass
    
    ## extract out the data from the file where we know there may be damage 
    for dmg in row['Dmg'].split('+'):
        print(f'{dmg.strip()}.*?(?= damage)')
        matches = re.findall(f'{dmg.strip()}.*?(?= damage)',f)
        print('{}\n\tdamage:{}'.format(row['Name'],matches))

2d8.*?(?= damage)
Chaos Bolt
	damage:['2d8 + 1d6']
1d6.*?(?= damage)
Chaos Bolt
	damage:['1d6', '1d6 extra']
1d4.*?(?= damage)
Magic Missile
	damage:['1d4 + 1 force']
1 (x3).*?(?= damage)
Magic Missile
	damage:[]
2d8.*?(?= damage)
Ice Storm
	damage:['2d8 bludgeoning']
4d6.*?(?= damage)
Ice Storm
	damage:['4d6 cold']
5d6.*?(?= damage)
Destructive Wave
	damage:['5d6 thunder', '5d6 radiant or necrotic']
5d6.*?(?= damage)
Destructive Wave
	damage:['5d6 thunder', '5d6 radiant or necrotic']
4d6.*?(?= damage)
Flame Strike
	damage:['4d6 fire', '4d6 radiant']
4d6.*?(?= damage)
Flame Strike
	damage:['4d6 fire', '4d6 radiant']
10d6.*?(?= damage)
Disintegrate
	damage:['10d6 + 40 force']
40.*?(?= damage)
Disintegrate
	damage:['40 force']
7d8.*?(?= damage)
Finger of Death
	damage:['7d8 + 30 necrotic']
30.*?(?= damage)
Finger of Death
	damage:['30 necrotic']
20d6.*?(?= damage)
Meteor Swarm
	damage:['20d6 fire', '20d6 bludgeoning']
20d6.*?(?= damage)
Meteor Swarm
	damage:['20d6 fire', '20d6 bludgeonin

In [11]:
dmg_spells_copy['dmgCpy'] = dmg_spells_copy['Dmg'].str.findall(r'\d+d\d+')
# dmg_spells_copy['dmg_die_count'] = np.select(
#     [dmg_spells_copy['dmgCpy'].str.contains(',')],
#     [None],
#     default = dmg_spells_copy['dmgCpy'].str.split('d')[0] 
# )
# dmg_spells_copy['dmg_die_size'] = np.select(
#     [dmg_spells_copy['dmgCpy'].str.contains(',')],
#     [None],
#     default = dmg_spells_copy['dmgCpy'].str.split('d')
# )

type(dmg_spells_copy['dmgCpy'].loc[0])
d1,d2 = [],[]
for i, row in dmg_spells_copy.iterrows():
    if len(row['dmgCpy']) == 1:
        d = row['dmgCpy'][0].split('d')
        d1.append(d[0])
        d2.append(d[1])
    else:
        d1.append(None)
        d2.append(None)

dmg_spells_copy['dmg_die_count'] = d1
dmg_spells_copy['dmg_die_size'] = d2
        
dmg_spells_copy.drop(columns=['dmgCpy'],inplace=True)
dmg_spells_copy['uc'] = np.select(
    [~dmg_spells_copy['Upcast'].isna()],[True],default=False
)
dmg_spells_copy

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,desc,upcast,lvlInc,mod,dmgDict,multDmg,hardAdd,addSCMod,multiCast,dmg_die_count,dmg_die_size,uc
0,0,Acid Splash,DEX,1d6,Acid,True,None,You hurl a bubble of acid. Choose one creatur...,None,"[{5: {2: 6}}, {11: {3: 6}}, {17: {4: 6}}]",False,None,None,None,None,None,1,6,False
1,0,Booming Blade,None,1d8,Thunder,True,None,You brandish the weapon used in the spell’s c...,None,None,False,None,None,None,None,None,1,8,False
2,0,Chill Touch,None,1d8,Necrotic,True,None,"You create a ghostly, skeletal hand in the sp...",None,"[{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]",False,None,None,None,None,None,1,8,False
3,0,Create Bonfire,DEX,1d8,Fire,True,None,You create a bonfire on ground that you can s...,None,"[{5: {2: 8}}, {11: {3: 8}}, {17: {4: 8}}]",False,None,None,None,None,None,1,8,False
4,0,Eldritch Blast,None,1d10,Force,True,None,A beam of crackling energy streaks toward a c...,None,None,False,None,None,None,None,None,1,10,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,9,Psychic Scream,INT,14d6,Psychic,False,None,You unleash the power of your mind to blast t...,None,None,False,None,None,None,None,None,14,6,False
172,9,Ravenous Void,STR,5d10,Force,False,None,You create a 20-foot-radius sphere of destruc...,None,None,False,None,None,None,None,None,5,10,False
173,9,Storm of Vengeance,CON,2d6 / 1d6 / 10d6 / 2d6 / 1d6,"Acid, Bludgeoning, Cold, Lightning, Thunder",False,Dice,"A churning storm cloud forms, centered on a p...",None,None,False,"[{'Acid': {1: 6}}, {'Bludgeoning': {2: 6}}, {'...",None,None,None,None,None,None,True
174,9,Time Ravage,CON,10d12,Necrotic,False,None,You target a creature you can see within rang...,None,None,False,None,None,None,None,None,10,12,False


In [ ]:
#these spells list an upcast, but don't have an upcast dictionary
dmg_spells_copy.loc[(dmg_spells_copy['uc']==True)&(dmg_spells_copy['upcast'].isna())]

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,desc,upcast,lvlInc,mod,dmgDict,multDmg,hardAdd,addSCMod,multiCast,dmg_die_count,dmg_die_size,uc
27,1,Armor of Agathys,None,5,Cold,False,Amount/Amount,"A protective magical force surrounds you, man...",None,None,False,None,None,None,None,None,None,None,True
30,1,Catapult,DEX,3d8,Bludgeoning,False,Other,Choose one object weighing 1 to 5 pounds with...,None,None,False,None,None,None,None,None,3,8,True
41,1,Hex,None,1d6,Necrotic,False,Duration,You place a curse on a creature that you can ...,None,None,False,None,None,None,None,None,1,6,True
42,1,Hunter's Mark,None,1d6,Variable,False,Duration,You choose a creature you can see within rang...,None,None,False,None,None,None,None,None,1,6,True
50,1,Tasha's Caustic Brew,DEX,2d4,Acid,False,Dice,A stream of acid emanates from you in a line ...,None,None,False,None,None,None,None,None,2,4,True
59,2,Cordon of Arrows,DEX,1d6,Piercing,False,Number,You plant four pieces of nonmagical ammunitio...,None,None,False,None,None,None,None,None,1,6,True
62,2,Flame Blade,None,3d6,Fire,False,Dice,You evoke a fiery blade in your free hand. Th...,None,None,False,None,None,None,None,None,3,6,True
72,2,Shadow Blade,None,2d8,Psychic,False,Dice,You weave together threads of shadow to creat...,None,None,False,None,None,None,None,None,2,8,True
75,2,Spiritual Weapon,None,1d8,Force,False,Dice,"You create a floating, spectral weapon within...",None,None,True,None,None,None,None,None,1,8,True
78,3,Ashardalon's Stride,None,1d6,Fire,False,Amount/Dice,The billowing flames of a dragon blast from y...,None,None,False,None,None,None,None,None,1,6,True


In [ ]:
dmg_spells_copy.loc[dmg_spells_copy['lvlInc'].isna()]
# dmg_spells_copy['dmg_die_count'] = dmg_spells_copy['dmg_die_count'].astype(int)
# dmg_spells_copy['dmg_die_size'] = dmg_spells_copy['dmg_die_size'].astype(int)
for i, row in dmg_spells_copy.iterrows():
    if not isinstance(row['lvlInc'],dict):
        if isinstance(row['dmgDict'],dict) and isinstance(row['dmg_die_count']) and isinstance(row['dmg_die_size']):
            d = row['dmgDict']
            d[1] = {row['dmg_die_count']:row['dmg_die_size']}
            row['lvlInc'] = d
        elif isinstance(row['dmg_die_count'],int) and isinstance(row['dmg_die_size']):
            row['lvlInc'] = {1:{row['dmg_die_count']:row['dmg_die_size']}}

# dmg_spells_copy['dmg_die_count'].dtype

dtype('O')

In [7]:
dmg_spells_copy.to_csv('./data/spells.csv',index=False)

column = [
    {CHAR LEVEL:[
        {1: [
            {Cold : {1 : 6}}, 
            {Piercing : {2 : 8}}
        ]
        },
        {5: [
            {Cold : {1 : 6}}, 
            {Piercing : {2 : 8}}
            ]
        },
        {11: [
            {Cold : {1 : 6}}, 
            {Piercing : {2 : 8}}
            ]
        },
        {17: [
            {Cold : {1 : 6}}, 
            {Piercing : {2 : 8}}
            ]
        }
    },

    {UPCAST:
    {
    Cold : {1 : 6},
    Piercing : {2 : 8}
    }

############################################################

DMG = [
        {
            1:[
                {Cold : {1 : 6}}, 
                {Piercing : {2 : 8}}
            ]
        }
        {
            5: [
                {Cold : {2 : 6}}, 
                {Piercing : {2 : 8}}
            ]
        },
        {
            11: [
                {Cold : {3 : 6}}, 
                {Piercing : {3: 8}}
            ]
        },
        {
            17: [
                {Cold : {4 : 6}}, 
                {Piercing : {4 : 8}}
            ]
        }
    }
]

UPCAST:
    [
        {DMG_TYPE:{min_level:{dice qty: die size}}},
        {DMG_TYPE:{min_level:{dice qty: die size}}}
    ]

dmg: {
    'lvl#':{'dmgtype':{qty:size}},
    'upcast':{'dmgtype':{qty:size}}
}

hard_add - int

multiple - intsle

The spell can have multiple damage types 

In [2]:
import pandas as pd
dmg_spells_copy=pd.read_csv('./data/spells.csv')

## Handle the base case

* One Damage Type

* One damage die (size/qty)



## Handle spell modifier

+SAM - create a boolean column

## Handle multiple damage types

{
    'type':{#:size}
}


In [287]:
import re
from bs4 import BeautifulSoup as BS

baseup = dmg_spells_copy.loc[
    (~dmg_spells_copy['Dmg'].str.contains('/|\+')) &
    (~dmg_spells_copy['Upcast'].isna()) &
    (dmg_spells_copy['HigherLvlIncrease']==False)
].copy()

higher_inc = dmg_spells_copy.loc[
    (~dmg_spells_copy['Dmg'].str.contains('/|\+')) &
    ((dmg_spells_copy['Upcast'].isna()) |
    (dmg_spells_copy['HigherLvlIncrease']==True))
].copy()

higher_inc_upcast = higher_inc.loc[
    higher_inc['HigherLvlIncrease']==False
].copy()

higher_inc_chr = higher_inc.loc[
    higher_inc['HigherLvlIncrease']==True
].copy()

multidmg = dmg_spells_copy.loc[
    (dmg_spells_copy['Dmg'].str.contains('/|\+'))
].copy()

adds_only = multidmg.loc[
    (multidmg['Dmg'].str.contains('\+')) &
    (~multidmg['Dmg'].str.contains('/'))
].copy()

slashes_only = multidmg.loc[
    (multidmg['Dmg'].str.contains('/')) & 
    (~multidmg['Dmg'].str.contains('\+'))
].copy()

add_slash = multidmg.loc[
    (multidmg['Dmg'].str.contains('/')) & 
    (multidmg['Dmg'].str.contains('\+'))
].copy()

display(
    baseup,
    higher_inc,
    adds_only,
    slashes_only,
    add_slash
)
display(
len(pd.concat([baseup,higher_inc,adds_only,slashes_only,add_slash])),
len(dmg_spells_copy))

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy
26,1,Absorb Elements,None,1d6,Variable,False,Dice,1d6
27,1,Armor of Agathys,None,5,Cold,False,Amount/Amount,5
28,1,Arms of Hadar,STR,2d6,Necrotic,False,Dice,2d6
29,1,Burning Hands,DEX,3d6,Fire,False,Dice,3d6
30,1,Catapult,DEX,3d8,Bludgeoning,False,Other,3d8
...,...,...,...,...,...,...,...,...
137,6,Circle of Death,CON,8d6,Necrotic,False,Dice,8d6
146,6,Otiluke's Freezing Sphere,CON,10d6,Cold,False,Dice,10d6
149,6,Wall of Thorns,DEX,7d8,"Piercing, Slashing",False,Dice,7d8
150,7,Crown of Stars,None,4d12,Radiant,False,Number,4d12


,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy
0,0,Acid Splash,DEX,1d6,Acid,True,None,1d6
1,0,Booming Blade,None,1d8,Thunder,True,None,1d8
2,0,Chill Touch,None,1d8,Necrotic,True,None,1d8
3,0,Create Bonfire,DEX,1d8,Fire,True,None,1d8
4,0,Eldritch Blast,None,1d10,Force,True,None,1d10
...,...,...,...,...,...,...,...,...
168,9,Blade of Disaster,None,4d12,Force,False,None,4d12
171,9,Psychic Scream,INT,14d6,Psychic,False,None,14d6
172,9,Ravenous Void,STR,5d10,Force,False,None,5d10
174,9,Time Ravage,CON,10d12,Necrotic,False,None,10d12


,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy
10,0,Magic Stone,None,1d6+SAM,Bludgeoning,True,None,1d6+SAM
18,0,Shillelagh,None,1d8+SAM,Bludgeoning,False,None,1d8+SAM
31,1,Chaos Bolt,None,2d8+1d6,Variable,False,Dice,2d8+1d6
46,1,Magic Missile,None,1d4+1 (x3),Force,False,Number,1d4+1 (x3)
75,2,Spiritual Weapon,None,1d8+SAM,Force,False,Dice,1d8+SAM
106,4,Ice Storm,DEX,2d8+4d6,"Bludgeoning, Cold",False,Dice,2d8+4d6
121,5,Destructive Wave,CON,5d6+5d6,"Thunder, Variable",False,None,5d6+5d6
123,5,Flame Strike,DEX,4d6+4d6,"Fire, Radiant",False,Dice,4d6+4d6
138,6,Disintegrate,DEX,10d6+40,Force,False,Dice,10d6+40
153,7,Finger of Death,CON,7d8+30,Necrotic,False,None,7d8+30


,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy
23,0,Toll the Dead,WIS,1d8 / 1d12,Necrotic,True,None,1d8 / 1d12
43,1,Ice Knife,DEX,1d10 / 2d6,"Cold, Piercing",False,Dice,1d10 / 2d6
49,1,Searing Smite,CON,1d6 / 1d6,Fire,False,Dice,1d6 / 1d6
65,2,Maximilian's Earthen Grasp,STR,2d6 / 2d6,Bludgeoning,False,None,2d6 / 2d6
66,2,Melf's Acid Arrow,None,4d4 / 2d4,Acid,False,Dice,4d4 / 2d4
87,3,Hunger of Hadar,DEX,2d6 / 2d6,"Acid, Cold",False,None,2d6 / 2d6
89,3,Lightning Arrow,DEX,4d8 / 2d8,Lightning,False,Dice,4d8 / 2d8
113,4,Storm Sphere,STR,2d6 / 4d6,"Bludgeoning, Lightning",False,None,2d6 / 4d6
114,4,Vitriolic Sphere,DEX,10d4 / 5d4,Acid,False,Dice,10d4 / 5d4
124,5,Holy Weapon,CON,2d8 / 4d8,Radiant,False,None,2d8 / 4d8


,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy
117,5,Bigby's Hand,None,4d8 / 2d6+SAM,"Bludgeoning, Force",False,Dice,4d8 / 2d6+SAM


176

176

In [288]:
baseup

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy
26,1,Absorb Elements,None,1d6,Variable,False,Dice,1d6
27,1,Armor of Agathys,None,5,Cold,False,Amount/Amount,5
28,1,Arms of Hadar,STR,2d6,Necrotic,False,Dice,2d6
29,1,Burning Hands,DEX,3d6,Fire,False,Dice,3d6
30,1,Catapult,DEX,3d8,Bludgeoning,False,Other,3d8
...,...,...,...,...,...,...,...,...
137,6,Circle of Death,CON,8d6,Necrotic,False,Dice,8d6
146,6,Otiluke's Freezing Sphere,CON,10d6,Cold,False,Dice,10d6
149,6,Wall of Thorns,DEX,7d8,"Piercing, Slashing",False,Dice,7d8
150,7,Crown of Stars,None,4d12,Radiant,False,Number,4d12


In [289]:
baseup['hard_mod'] = 0
baseup['mult'] = 1
baseup['upcast'] = None
baseup['dmg'] = None
for i,row in baseup.iterrows():
    if 'd' not in row['Dmg']:
        #hard add damage
        baseup.loc[i,'hard_mod'] = row['Dmg']
    if '(x' in row['Dmg']:
        x=re.findall('(?<=\(x)(\d+)(?=\))',row['Dmg'])
        baseup.loc[i,'mult']= int(x[0])
        dmg = row['Dmg'].split('(')[0].split('d')
        print(row['Dmg'],dmg)
        baseup.at[i,'dmg'] = {1:{row['DmgType']:[{int(dmg[0]):int(dmg[1])}]}}
    if row['Upcast'] is not None:
        try:
            f = open('./data_for_parsing/spells/{}.html'.format(row['Name']),'r').read()
        except: 
            print("couldnt find file: {}.html".format(row['Name']))
            f=''
        soup = re.sub('\n+','\n',BS(f).get_text())
        upcast = re.findall('(?<=using a spell slot).*?(?=for each slot level above)',soup)
        if len(upcast) > 0:
            min_lvl = re.findall(r'(\d)?(?=nd|st|th|rd)',upcast[0])
            dice = re.findall(r'(\d)+d(\d+)',upcast[0])
            # print(min_lvl,dice)
            if len(min_lvl)>0 and len(dice) >0:
                uc = {int(min_lvl[0]):{int(dice[0][0]):int(dice[0][1])}}
                # print(uc)
                baseup.at[i,'upcast'] = uc

    if len(row['DmgType'].split(', ')) > 1:
        try:
            f = open('./data_for_parsing/spells/{}.html'.format(row['Name']),'r').read()
        except:
            print("couldnt find file: {}.html".format(row['Name']))
        soup = re.sub('\n+','\n',BS(f).get_text())
        d = {1:{}}
        for t in row['DmgType'].split(', '):
            res = re.findall('(\d+d\d+) {}'.format(t),soup,re.IGNORECASE)
            if len(res) > 0:
                dice = res[0].split('d')
                d[1][t] = {int(dice[0]):int(dice[1])}
        print(d)
        baseup.at[i,'dmg'] = d
        

    if baseup.at[i,'dmg'] is None and 'd' in row['Dmg']:
        dice = row['Dmg'].split('d')
        baseup.at[i,'dmg'] = {1:{row['DmgType']:[{int(dice[0]):int(dice[1])}]}}
        # print(upcast)

baseup.head(50)

2d6 (x3) ['2', '6 ']
{1: {'Piercing': {7: 8}, 'Slashing': {7: 8}}}


,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy,hard_mod,mult,upcast,dmg
26,1,Absorb Elements,None,1d6,Variable,False,Dice,1d6,0,1,{2: {1: 6}},{1: {'Variable': [{1: 6}]}}
27,1,Armor of Agathys,None,5,Cold,False,Amount/Amount,5,5,1,None,None
28,1,Arms of Hadar,STR,2d6,Necrotic,False,Dice,2d6,0,1,{2: {1: 6}},{1: {'Necrotic': [{2: 6}]}}
29,1,Burning Hands,DEX,3d6,Fire,False,Dice,3d6,0,1,{2: {1: 6}},{1: {'Fire': [{3: 6}]}}
30,1,Catapult,DEX,3d8,Bludgeoning,False,Other,3d8,0,1,{2: {1: 8}},{1: {'Bludgeoning': [{3: 8}]}}
32,1,Chromatic Orb,None,3d8,Variable,False,Dice,3d8,0,1,{2: {1: 8}},{1: {'Variable': [{3: 8}]}}
33,1,Dissonant Whispers,WIS,3d6,Psychic,False,Dice,3d6,0,1,{2: {1: 6}},{1: {'Psychic': [{3: 6}]}}
35,1,Earth Tremor,DEX,1d6,Bludgeoning,False,Dice,1d6,0,1,{2: {1: 6}},{1: {'Bludgeoning': [{1: 6}]}}
36,1,Ensnaring Strike,STR,1d6,Piercing,False,Dice,1d6,0,1,{2: {1: 6}},{1: {'Piercing': [{1: 6}]}}
37,1,Frost Fingers,CON,2d8,Cold,False,Dice,2d8,0,1,{2: {1: 8}},{1: {'Cold': [{2: 8}]}}


### Containing a '+'

In [291]:
adds_only

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy
10,0,Magic Stone,None,1d6+SAM,Bludgeoning,True,None,1d6+SAM
18,0,Shillelagh,None,1d8+SAM,Bludgeoning,False,None,1d8+SAM
31,1,Chaos Bolt,None,2d8+1d6,Variable,False,Dice,2d8+1d6
46,1,Magic Missile,None,1d4+1 (x3),Force,False,Number,1d4+1 (x3)
75,2,Spiritual Weapon,None,1d8+SAM,Force,False,Dice,1d8+SAM
106,4,Ice Storm,DEX,2d8+4d6,"Bludgeoning, Cold",False,Dice,2d8+4d6
121,5,Destructive Wave,CON,5d6+5d6,"Thunder, Variable",False,None,5d6+5d6
123,5,Flame Strike,DEX,4d6+4d6,"Fire, Radiant",False,Dice,4d6+4d6
138,6,Disintegrate,DEX,10d6+40,Force,False,Dice,10d6+40
153,7,Finger of Death,CON,7d8+30,Necrotic,False,None,7d8+30


In [292]:
adds_only['addspellmod'] = [False]*len(adds_only)
adds_only['mult'] = [1]*len(adds_only)
adds_only['hard_mod'] = 0
adds_only['dmg'] = [None]*len(adds_only)
for i,row in adds_only.iterrows():
    print(row['Name'])
    if '+SAM' in row['Dmg']:
        print("add spellcasting modifier")
        adds_only.loc[i,'addspellmod'] = True
        print("\tSAM: ",True)
        dice = row['Dmg'].split('+')[0].split('d')
        adds_only.at[i,'dmg'] = {1:{row['DmgType']:[{int(dice[0]):int(dice[1])}]}}
    if '(x' in row['Dmg']:
        print('\tmultiple waves/beams/attacks')
        print(re.findall(r'\(x(\d)+\)',row['Dmg']))
        adds_only.loc[i,'mult'] = re.findall(r'\(x(\d)+\)',row['Dmg'])[0]
        dice = row['Dmg'].split('+')[0].split('d')
        adds_only.at[i,'dmg'] = {1:{row['DmgType']:[{int(dice[0]):int(dice[1])}]}}
    if len(re.findall(r'(?<=\+)(\d+)$',row['Dmg'])) > 0: #len(re.findall(r'(?<=\+)(\d+)(?!d)',row['Dmg'])) > 0:
        print('\thard addition')
        print(re.findall(r'(?<=\+)(\d+)(?!d)',row['Dmg']))
        adds_only.loc[i,'hard_mod'] = re.findall(r'(?<=\+)(\d+)(?!d)',row['Dmg'])[0]
        dice = row['Dmg'].split('+')[0].split('d')
        adds_only.at[i,'dmg'] = {1:{row['DmgType']:[{int(dice[0]):int(dice[1])}]}}
    if '+' in row['Dmg'] and not 'SAM' in row['Dmg'] and len(re.findall(r'(?<=\+)(\d+)$',row['Dmg'])) == 0:
        try:
            f = open("./data_for_parsing/spells/{}.html".format(row['Name'])).read()
        except:
            print("couldn't find file for {}".format(row['Name']))
            f = ''
        soup = re.sub('\n+','\n',BS(f).get_text().strip())
        dmgs = {}
        for ty in row['DmgType'].split(','):
            ty = ty.strip()
            # print(ty)
            res = re.findall('(\d+d\d+) {} damage'.format(ty),soup,re.IGNORECASE)
            # print(res)
            if len(res) > 0:
               res = res[0].split('d')
               dmgs[1] = {ty:[{int(res[0]):int(res[1])}]}
            
        print('\tmultiple damage types')
        print(dmgs)
        adds_only.at[i,'dmg'] = dmgs
        # dmgs = row['Dmg'].split('+')
        # typs = row['DmgType'].split(',')
        # adds_only.at[i,'dmg'] = {'dmgs':dmgs,'types':typs}
        # print({'dmgs':dmgs,'types':typs})
        
adds_only.loc[adds_only['mult'].isna(),'mult'] = 1
adds_only


Magic Stone
add spellcasting modifier
	SAM:  True
Shillelagh
add spellcasting modifier
	SAM:  True
Chaos Bolt
	multiple damage types
{}
Magic Missile
	multiple waves/beams/attacks
['3']
	multiple damage types
{}
Spiritual Weapon
add spellcasting modifier
	SAM:  True
Ice Storm
	multiple damage types
{1: {'Cold': [{4: 6}]}}
Destructive Wave
	multiple damage types
{1: {'Thunder': [{5: 6}]}}
Flame Strike
	multiple damage types
{1: {'Radiant': [{4: 6}]}}
Disintegrate
	hard addition
['40']
Finger of Death
	hard addition
['30']
Meteor Swarm
	multiple damage types
{1: {'Fire': [{20: 6}]}}


,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy,addspellmod,mult,hard_mod,dmg
10,0,Magic Stone,None,1d6+SAM,Bludgeoning,True,None,1d6+SAM,True,1,0,{1: {'Bludgeoning': [{1: 6}]}}
18,0,Shillelagh,None,1d8+SAM,Bludgeoning,False,None,1d8+SAM,True,1,0,{1: {'Bludgeoning': [{1: 8}]}}
31,1,Chaos Bolt,None,2d8+1d6,Variable,False,Dice,2d8+1d6,False,1,0,{}
46,1,Magic Missile,None,1d4+1 (x3),Force,False,Number,1d4+1 (x3),False,3,0,{}
75,2,Spiritual Weapon,None,1d8+SAM,Force,False,Dice,1d8+SAM,True,1,0,{1: {'Force': [{1: 8}]}}
106,4,Ice Storm,DEX,2d8+4d6,"Bludgeoning, Cold",False,Dice,2d8+4d6,False,1,0,{1: {'Cold': [{4: 6}]}}
121,5,Destructive Wave,CON,5d6+5d6,"Thunder, Variable",False,None,5d6+5d6,False,1,0,{1: {'Thunder': [{5: 6}]}}
123,5,Flame Strike,DEX,4d6+4d6,"Fire, Radiant",False,Dice,4d6+4d6,False,1,0,{1: {'Radiant': [{4: 6}]}}
138,6,Disintegrate,DEX,10d6+40,Force,False,Dice,10d6+40,False,1,40,{1: {'Force': [{10: 6}]}}
153,7,Finger of Death,CON,7d8+30,Necrotic,False,None,7d8+30,False,1,30,{1: {'Necrotic': [{7: 8}]}}


### Containing a "/"

In [293]:
slashes_only

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy
23,0,Toll the Dead,WIS,1d8 / 1d12,Necrotic,True,None,1d8 / 1d12
43,1,Ice Knife,DEX,1d10 / 2d6,"Cold, Piercing",False,Dice,1d10 / 2d6
49,1,Searing Smite,CON,1d6 / 1d6,Fire,False,Dice,1d6 / 1d6
65,2,Maximilian's Earthen Grasp,STR,2d6 / 2d6,Bludgeoning,False,None,2d6 / 2d6
66,2,Melf's Acid Arrow,None,4d4 / 2d4,Acid,False,Dice,4d4 / 2d4
87,3,Hunger of Hadar,DEX,2d6 / 2d6,"Acid, Cold",False,None,2d6 / 2d6
89,3,Lightning Arrow,DEX,4d8 / 2d8,Lightning,False,Dice,4d8 / 2d8
113,4,Storm Sphere,STR,2d6 / 4d6,"Bludgeoning, Lightning",False,None,2d6 / 4d6
114,4,Vitriolic Sphere,DEX,10d4 / 5d4,Acid,False,Dice,10d4 / 5d4
124,5,Holy Weapon,CON,2d8 / 4d8,Radiant,False,None,2d8 / 4d8


In [294]:
slashes_only['dmg'] = None
for i,row in slashes_only.iterrows():
    try:
        f = open("./data_for_parsing/spells/{}.html".format(row['Name'])).read()
    except:
        print("File for {} not found".format(row['Name']))
        pass
    soup = BS(f)
    content = soup('div',{'id':'page-content'})[0]
    content = content.get_text().strip()
    # print(row['Name'])
    dmg_d = {}
    for dmg in row['DmgType'].split(','):
        dmg = dmg.strip()
        res = re.findall('(\d+d\d+) {}'.format(dmg),content,re.IGNORECASE)
        dmg_dict = {dmg:[]}
        for r in res:
            dmg_dict[dmg].append({int(r.split('d')[0]):int(r.split('d')[1])})
        if row['HigherLvlIncrease']:
            high_lvl = re.findall('(?<=when you reach).*?(?=\.)',content)
        else:
            high_lvl = None
        # print("{}\n\t{}: {}\n\t{}".format(row['Name'],dmg,res,high_lvl))
        if row['Upcast'] is not None and row['Upcast']=='Dice':
            upcast = re.findall('(?<=using a spell slot).*?(?=for each slot level above)',content)
            if len(upcast) > 0:
                min_lvl = re.findall(r'(\d)?(?=nd|st|th|rd)',upcast[0])
                dice = re.findall(r'(\d)+d(\d+)',upcast[0])
        else:
            upcast = None
        # print("\t{}: \n\tlevel increase: {}\n\tupcast: {}\t{}".format(dmg_dict,high_lvl,upcast,min_lvl))
        dmg_d.update(dmg_dict)
    # print(row['Name'],dmg_d)
    # print(content.get_text().strip())
    # pass
    slashes_only.at[i,'dmg'] = {1:dmg_d}
slashes_only

File for Maximilian's Earthen Grasp not found


,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy,dmg
23,0,Toll the Dead,WIS,1d8 / 1d12,Necrotic,True,None,1d8 / 1d12,"{1: {'Necrotic': [{1: 8}, {1: 12}]}}"
43,1,Ice Knife,DEX,1d10 / 2d6,"Cold, Piercing",False,Dice,1d10 / 2d6,"{1: {'Cold': [{2: 6}], 'Piercing': [{1: 10}]}}"
49,1,Searing Smite,CON,1d6 / 1d6,Fire,False,Dice,1d6 / 1d6,"{1: {'Fire': [{1: 6}, {1: 6}]}}"
65,2,Maximilian's Earthen Grasp,STR,2d6 / 2d6,Bludgeoning,False,None,2d6 / 2d6,{1: {'Bludgeoning': []}}
66,2,Melf's Acid Arrow,None,4d4 / 2d4,Acid,False,Dice,4d4 / 2d4,"{1: {'Acid': [{4: 4}, {2: 4}]}}"
87,3,Hunger of Hadar,DEX,2d6 / 2d6,"Acid, Cold",False,None,2d6 / 2d6,"{1: {'Acid': [{2: 6}], 'Cold': [{2: 6}]}}"
89,3,Lightning Arrow,DEX,4d8 / 2d8,Lightning,False,Dice,4d8 / 2d8,"{1: {'Lightning': [{4: 8}, {2: 8}]}}"
113,4,Storm Sphere,STR,2d6 / 4d6,"Bludgeoning, Lightning",False,None,2d6 / 4d6,"{1: {'Bludgeoning': [{2: 6}], 'Lightning': [{4..."
114,4,Vitriolic Sphere,DEX,10d4 / 5d4,Acid,False,Dice,10d4 / 5d4,"{1: {'Acid': [{10: 4}, {5: 4}]}}"
124,5,Holy Weapon,CON,2d8 / 4d8,Radiant,False,None,2d8 / 4d8,"{1: {'Radiant': [{2: 8}, {4: 8}]}}"


In [295]:
higher_inc_chr['dmg'] = None
for i,row in higher_inc_chr.iterrows():
    try:
        f = open('./data_for_parsing/spells/{}.html'.format(row['Name']),'r').read()
    except:
        print("No file for {}".format(row['Name']))
        pass
    soup = BS(f).get_text().strip()
    soup = re.sub('\n+','\n',soup)
    res = re.findall('\d+.. level.*?(?=\.)',soup)
    base = row['Dmg'].split('d')
    dmg_dict = {1:{row['DmgType']:[{int(base[0]):int(base[1])}]}}
    if len(res) > 0:
        r1 = re.findall('(\d+)th level \((\d+)d(\d)\)',res[0])
        for t in r1:
            dmg_dict[int(t[0])] = {row['DmgType']:[{int(t[1]):int(t[2])}]}
    higher_inc_chr.at[i,'dmg'] = dmg_dict
higher_inc_chr

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy,dmg
0,0,Acid Splash,DEX,1d6,Acid,True,None,1d6,"{1: {'Acid': [{1: 6}]}, 5: {'Acid': [{2: 6}]},..."
1,0,Booming Blade,None,1d8,Thunder,True,None,1d8,{1: {'Thunder': [{1: 8}]}}
2,0,Chill Touch,None,1d8,Necrotic,True,None,1d8,"{1: {'Necrotic': [{1: 8}]}, 5: {'Necrotic': [{..."
3,0,Create Bonfire,DEX,1d8,Fire,True,None,1d8,"{1: {'Fire': [{1: 8}]}, 5: {'Fire': [{2: 8}]},..."
4,0,Eldritch Blast,None,1d10,Force,True,None,1d10,{1: {'Force': [{1: 10}]}}
5,0,Fire Bolt,None,1d10,Fire,True,None,1d10,{1: {'Fire': [{1: 10}]}}
6,0,Frostbite,CON,1d6,Cold,True,None,1d6,"{1: {'Cold': [{1: 6}]}, 5: {'Cold': [{2: 6}]},..."
7,0,Green-Flame Blade,None,1d8,Fire,True,None,1d8,{1: {'Fire': [{1: 8}]}}
8,0,Infestation,CON,1d6,Poison,True,None,1d6,"{1: {'Poison': [{1: 6}]}, 5: {'Poison': [{2: 6..."
9,0,Lightning Lure,STR,1d8,Lightning,True,None,1d8,"{1: {'Lightning': [{1: 8}]}, 5: {'Lightning': ..."


In [296]:
higher_inc_upcast['dmg'] = None
for i,row in higher_inc_upcast.iterrows():
    dmg_die = row['Dmg'].split('d')
    print(dmg_die,row['Dmg'])
    if len(dmg_die) == 2:
        higher_inc_upcast.at[i,'dmg'] = {1:{row['DmgType']:[{int(dmg_die[0]):int(dmg_die[1])}]}}
higher_inc_upcast

['1', '4'] 1d4
['2', '8'] 2d8
['2', '6'] 2d6
['1', '6'] 1d6
['1', '8'] 1d8
['1', '6'] 1d6
['3', '8'] 3d8
['1', '4'] 1d4
['6', '6'] 6d6
['4', '8'] 4d8
['3', '8'] 3d8
['2', '8'] 2d8
['3', '6'] 3d6
['2', '8'] 2d8
['5', '10'] 5d10
['20'] 20
['4', '8'] 4d8
['2', '8'] 2d8
['4', '10'] 4d10
['4', '6'] 4d6
['5', '10'] 5d10
['4', '10'] 4d10
['6', '6'] 6d6
['5', '12'] 5d12
['6', '10'] 6d10
['8', '6'] 8d6
['4', '8'] 4d8
['6', '10'] 6d10
['5', '10'] 5d10
['8', '8'] 8d8
['14', '6'] 14d6
['4', '6'] 4d6
['2', '10'] 2d10
['5', '10'] 5d10
['6', '8'] 6d8
['6', '8'] 6d8
['7', '10'] 7d10
['3', '10'] 3d10
['10', '6'] 10d6
['10', '10'] 10d10
['10', '6'] 10d6
['12', '8'] 12d8
['8', '10'] 8d10
['5', '6'] 5d6
['4', '6'] 4d6
['7', '6'] 7d6
['10', '8'] 10d8
['8', '8'] 8d8
['12', '6'] 12d6
['4', '12'] 4d12
['14', '6'] 14d6
['5', '10'] 5d10
['10', '12'] 10d12
['4', '10'] 4d10


,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy,dmg
34,1,Divine Favor,CHR,1d4,Radiant,False,None,1d4,{1: {'Radiant': [{1: 4}]}}
47,1,Magnify Gravity,CON,2d8,Force,False,None,2d8,{1: {'Force': [{2: 8}]}}
51,1,Thunderous Smite,STR,2d6,Thunder,False,None,2d6,{1: {'Thunder': [{2: 6}]}}
54,1,Wrathful Smite,WIS,1d6,Psychic,False,None,1d6,{1: {'Psychic': [{1: 6}]}}
55,1,Zephyr Strike,None,1d8,Force,False,None,1d8,{1: {'Force': [{1: 8}]}}
69,2,Phantasmal Force,INT,1d6,Psychic,False,None,1d6,{1: {'Psychic': [{1: 6}]}}
79,3,Blinding Smite,CON,3d8,Radiant,False,None,3d8,{1: {'Radiant': [{3: 8}]}}
81,3,Crusader's Mantle,None,1d4,Radiant,False,None,1d4,{1: {'Radiant': [{1: 4}]}}
92,3,Pulse Wave,CON,6d6,Force,False,None,6d6,{1: {'Force': [{6: 6}]}}
96,3,Tidal Wave,DEX,4d8,Bludgeoning,False,None,4d8,{1: {'Bludgeoning': [{4: 8}]}}


In [297]:
display(add_slash)
f = open('./data_for_parsing/spells/Bigby\'s Hand.html','r').read()
soup = re.sub('\n+','\n',BS(f).get_text())
re.findall('(\d+d\d+).*slot level',soup)
row = add_slash.iloc[0]
for t in row['DmgType'].split(', '):
    res = re.findall('(\d+d\d+) {}'.format(t),soup,re.IGNORECASE)
    print(res)

add_slash['dmg'] = None 
add_slash.at[117,'dmg'] = {1:{"Bludgeoning":[{2:6}],"Force":[{4:8}]}}
add_slash['upcast'] = None 
add_slash.at[117,'upcast'] = {6:{"Force":[{2:8}]},5:{"Bludgeoning":[{2:6}]}}
add_slash.at[117,'addspellmod'] = True
add_slash

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy
117,5,Bigby's Hand,None,4d8 / 2d6+SAM,"Bludgeoning, Force",False,Dice,4d8 / 2d6+SAM


[]
['4d8']


,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy,dmg,upcast,addspellmod
117,5,Bigby's Hand,None,4d8 / 2d6+SAM,"Bludgeoning, Force",False,Dice,4d8 / 2d6+SAM,"{1: {'Bludgeoning': [{2: 6}], 'Force': [{4: 8}]}}","{6: {'Force': [{2: 8}]}, 5: {'Bludgeoning': [{...",True


In [298]:
fr = pd.concat([
    baseup,
    add_slash,
    slashes_only,
    adds_only,
    higher_inc_chr,
    higher_inc_upcast
]).sort_index()
fr['mult'].fillna(1,inplace=True)
fr['hard_mod'].fillna(0,inplace=True)
fr['addspellmod'].fillna(False,inplace=True)
fr['addspellmod'] = fr['addspellmod'].astype(bool)
fr.drop(columns=['Dmg','dmgCpy'],inplace=True)

In [300]:
for i,row in fr.iterrows():
    if row['upcast'] is None:
        fr.at[i,'upcast'] = {}
    if row['dmg'] is None:
        fr.at[i,'dmg'] = {}
fr

,Lvl,Name,Save,DmgType,HigherLvlIncrease,Upcast,hard_mod,mult,upcast,dmg,addspellmod
0,0,Acid Splash,DEX,Acid,True,None,0,1,NaN,"{1: {'Acid': [{1: 6}]}, 5: {'Acid': [{2: 6}]},...",False
1,0,Booming Blade,None,Thunder,True,None,0,1,NaN,{1: {'Thunder': [{1: 8}]}},False
2,0,Chill Touch,None,Necrotic,True,None,0,1,NaN,"{1: {'Necrotic': [{1: 8}]}, 5: {'Necrotic': [{...",False
3,0,Create Bonfire,DEX,Fire,True,None,0,1,NaN,"{1: {'Fire': [{1: 8}]}, 5: {'Fire': [{2: 8}]},...",False
4,0,Eldritch Blast,None,Force,True,None,0,1,NaN,{1: {'Force': [{1: 10}]}},False
...,...,...,...,...,...,...,...,...,...,...,...
171,9,Psychic Scream,INT,Psychic,False,None,0,1,NaN,{1: {'Psychic': [{14: 6}]}},False
172,9,Ravenous Void,STR,Force,False,None,0,1,NaN,{1: {'Force': [{5: 10}]}},False
173,9,Storm of Vengeance,CON,"Acid, Bludgeoning, Cold, Lightning, Thunder",False,Dice,0,1,NaN,"{1: {'Acid': [{1: 6}], 'Bludgeoning': [{2: 6}]...",False
174,9,Time Ravage,CON,Necrotic,False,None,0,1,NaN,{1: {'Necrotic': [{10: 12}]}},False


In [301]:
fr['time'] = None
fr['range'] = None
fr['components'] = None
fr['duration'] = None
fr['classes'] = None
fr['conc'] = False
for i,row in fr.iterrows():
    try:
        f = open('./data_for_parsing/spells/{}.html'.format(row['Name']),'r').read()
    except:
        print("No file for {}".format(row['Name']))
        pass
    soup = re.sub('\n+','\n',BS(f).getText())
    soup = re.sub('\t+','',soup)
    soup = re.sub(' +',' ',soup)
    soup = re.sub('\n \n','\n',soup)
    cast_time = re.findall('(?<=Casting Time).*?(?=Range)',soup,re.DOTALL)
    if len(cast_time) == 1:
        cast_time = re.sub('\n|:|','',cast_time[0])
        fr.loc[i,'time'] = cast_time
    rng = re.findall('(?<=Range).*?(?=Components)',soup,re.DOTALL)
    print(len(rng))
    if len(rng) == 1:
        rng = re.sub('\n|\:','',rng[0])
        fr.loc[i,'range'] = rng
    dur = re.findall('(?<=Duration\:\n ).*?(?=\n)',soup)
    if len(dur) == 1:
        dur = re.sub('\n|:|','',dur[0])
        fr.loc[i,'duration'] = dur
    comp = re.findall('(?<=Components).*?(?=Duration)',soup,re.DOTALL)
    if len(comp) == 1:
        comp = re.sub('\n|:|','',comp[0])
        fr.loc[i,'components'] = comp
    
    classes = re.findall('(?<=Spell Lists\.\n ).*?(?=Help)',soup,re.DOTALL)
    if len(classes) > 0:
        classes = classes[0]
        cl = list(map(lambda x: x if len(re.findall('[A-Z]',x)) > 0 else '',classes.split('\n')))
        res = []
        for c in cl:
            if len(c) > 0:
                c = re.sub('\(.*\)','',c)
                c = c.strip()
                res.append(c)
        fr.at[i,'classes'] = res


fr.loc[fr['duration'].str.contains('Concentration'),'conc'] = True
fr

    # extract the descriptions, casting times, range, components, and duration...

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
No file for Maximilian's Earthen Grasp
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


,Lvl,Name,Save,DmgType,HigherLvlIncrease,Upcast,hard_mod,mult,upcast,dmg,addspellmod,time,range,components,duration,classes,conc
0,0,Acid Splash,DEX,Acid,True,None,0,1,NaN,"{1: {'Acid': [{1: 6}]}, 5: {'Acid': [{2: 6}]},...",False,1 action,60 feet,"V, S",Instantaneous,"[Artificer, Sorcerer, Wizard]",False
1,0,Booming Blade,None,Thunder,True,None,0,1,NaN,{1: {'Thunder': [{1: 8}]}},False,1 action,Self (5-foot radius),"S, M (a melee weapon worth at least 1 sp)",1 round,"[Artificer, Sorcerer, Warlock, Wizard]",False
2,0,Chill Touch,None,Necrotic,True,None,0,1,NaN,"{1: {'Necrotic': [{1: 8}]}, 5: {'Necrotic': [{...",False,1 action,120 feet,"V, S",1 round,"[Sorcerer, Warlock, Wizard]",False
3,0,Create Bonfire,DEX,Fire,True,None,0,1,NaN,"{1: {'Fire': [{1: 8}]}, 5: {'Fire': [{2: 8}]},...",False,1 action,60 feet,"V, S","Concentration, up to 1 minute","[Artificer, Druid, Sorcerer, Warlock, Wizard]",True
4,0,Eldritch Blast,None,Force,True,None,0,1,NaN,{1: {'Force': [{1: 10}]}},False,1 action,120 feet,"V, S",Instantaneous,[Warlock],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,9,Psychic Scream,INT,Psychic,False,None,0,1,NaN,{1: {'Psychic': [{14: 6}]}},False,1 action,90 feet,S,Instantaneous,"[Bard, Sorcerer, Warlock, Wizard]",False
172,9,Ravenous Void,STR,Force,False,None,0,1,NaN,{1: {'Force': [{5: 10}]}},False,1 action,"1,000 feet","V, S, M (a small, nine-pointed star made of i...","Concentration, up to 1 minute",[Wizard],True
173,9,Storm of Vengeance,CON,"Acid, Bludgeoning, Cold, Lightning, Thunder",False,Dice,0,1,NaN,"{1: {'Acid': [{1: 6}], 'Bludgeoning': [{2: 6}]...",False,1 action,Sight,"V, S","Concentration, up to 1 minute",[Druid],True
174,9,Time Ravage,CON,Necrotic,False,None,0,1,NaN,{1: {'Necrotic': [{10: 12}]}},False,1 action,90 feet,"V, S, M (an hourglass filled with diamond dus...",Instantaneous,[Wizard],False


In [ ]:
fr['desc'] = ''
for i,row in fr.iterrows():
    try:
        f = open('./data_for_parsing/spells/{}.html'.format(row['Name']),'r').read()
        soup = BS(f)
        desc = soup.find('div',{'id':'page-content'})
        desc = desc.getText()
        desc = re.sub(' +',' ',desc)
        desc = re.sub('\n+','\n',desc)
        desc = re.sub('\n \n','\n',desc)
        desc = re.sub('’','\'\'',desc)
        # desc = re.sub('\n ?(?=[A-Z])',' ',desc)
        desc = re.sub('\n ,',',',desc)
        desc = desc.strip()
        fr.loc[i,'desc'] = desc
    except:
        print("No file for {}".format(row['Name']))
        f=''
        pass        
    #extract the description


No file for Maximilian's Earthen Grasp


In [327]:
fr.loc[0,'desc']

"Source: Player's Handbook\n Conjuration cantrip\n Casting Time:\n 1 action\n Range:\n 60 feet\n Components:\n V, S\n Duration:\n Instantaneous\n You hurl a bubble of acid. Choose one creature you can see within range, or choose two creatures you can see within range that are within 5 feet of each other. A target must succeed on a Dexterity saving throw or take 1d6 acid damage.\n At Higher Levels.\n This spell's damage increases by 1d6 when you reach 5th level (2d6), 11th level (3d6), and 17th level (4d6).\n Spell Lists.\n Artificer,\n Sorcerer,\n Wizard"

In [328]:
fr.to_csv('./data/parsed_spells.csv',index=False)

## Handle Per-Turn Effects vs Base Effects

## Handle multiple casts in one turn

## Handle Upcasts

## Handle Level Increases

## Combine and Merge all Frames

## Export to CSV

## Script Database Upload